In [2]:
import numpy as np
import pandas as pd
import sys
from keras.models import Sequential
from keras.layers import LSTM, Activation, Flatten, Dropout, Dense, Embedding, TimeDistributed, CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
dataset = pd.read_csv('taylor_swift_lyrics.csv', encoding = 'latin1')

dataset.head()

,artist,album,track_title,track_n,lyric,line,year
0,Taylor Swift,Taylor Swift,Tim McGraw,1,He said the way my blue eyes shined,1,2006
1,Taylor Swift,Taylor Swift,Tim McGraw,1,Put those Georgia stars to shame that night,2,2006
2,Taylor Swift,Taylor Swift,Tim McGraw,1,"I said, ""That's a lie""",3,2006
3,Taylor Swift,Taylor Swift,Tim McGraw,1,Just a boy in a Chevy truck,4,2006
4,Taylor Swift,Taylor Swift,Tim McGraw,1,That had a tendency of gettin' stuck,5,2006


In [4]:
def processFirtLine(lyrics, songID, songName, row):
    lyrics.append(row['lyric'] + '\n')
    songID.append(row['year']*100 + row['track_n'])
    songName.append(row['track_title'])
    return lyrics, songID, songName

lyrics = []
songID = []
songName = []

songNumber = 1
i = 0
isFirstLine = True

for index,row in dataset.iterrows():
    if(songNumber == row['track_n']):
        if(isFirstLine):
            lyrics, songID, songName = processFirtLine(lyrics, songID, songName, row)
            isFirstLine = False
        else:
            lyrics[i] += row['lyric'] + '\n'
    
    else:
        lyrics, songID, songName = processFirtLine(lyrics, songID, songName, row)
        songNumber = row['track_n']
        i += 1

In [7]:
lyrics_data = pd.DataFrame({'songID':songID, 'songName':songName, 'lyrics':lyrics})

with open('lyricsText.txt', 'w', encoding= 'utf-8') as filehandle:
    for listitem in lyrics:
        filehandle.write('%s\n' % listitem)

textFileName = 'lyricsText.txt'
raw_text = open(textFileName, encoding = 'UTF-8').read()
raw_text = raw_text.lower()


In [8]:
chars = sorted(list(set(raw_text)))
int_chars = dict((i, c) for i, c in enumerate(chars))
chars_int = dict((i, c) for c, i in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)

In [9]:
print('Total Characters: ', n_chars)
print('Total Vocab:', n_vocab)

Total Characters:  173698
Total Vocab: 58


In [12]:
seq_len = 100
data_X = []
data_Y = []

for i in range (0, n_chars - seq_len, 1):
    seq_in = raw_text[i:i + seq_len]
    seq_out = raw_text[i + seq_len]
    data_X.append([chars_int[char] for char in seq_in])
    data_Y.append(chars_int[seq_out])

n_patterns = len(data_X)
print('Total Patterns:', n_patterns)

Total Patterns: 173598


In [13]:
X = np.reshape(data_X, (n_patterns, seq_len, 1))
X = X/float(n_vocab)
Y = np_utils.to_categorical(data_Y)

Building the Modelo

In [14]:
LSTM_layer_num = 4
layer_size = [256, 256, 256, 256]
model = Sequential()

In [16]:
model.add(CuDNNLSTM(layer_size[0], input_shape = (X.shape[1], X.shape[2]), return_sequences = True))

for i in range(1, LSTM_layer_num):
    model.add(CuDNNLSTM(layer_size[i], return_sequences=True))

model.add(Flatten())
model.add(Dense(Y.shape[1]))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam')

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 100, 256)          265216    
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 100, 256)          526336    
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 100, 256)          526336    
                                                                 
 cu_dnnlstm_3 (CuDNNLSTM)    (None, 100, 256)          526336    
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 58)                1484858   
                                                                 
 activation (Activation)     (None, 58)                0

In [17]:
checkpoinmpon45j3_name = 'Weights-LSTM-improvement-{epoch:03d}-{loss:.5f}-bigger.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='loss', verbose=1, save_best_only=True, mode='min')
callback_list = [checkpoint]

In [19]:
model_params = {'epochs':30,
                'batch_size':128,
                'callbacks':callback_list,
                'verbose':1,
                'validation_split':0.2,
                'validation_data':None,
                'shuffle':True,
                'initial_epoch':0,
                'steps_per_epoch':None,
                'validation_steps':None}

model.fit(X,Y,
          epochs = model_params['epochs'],
          batch_size = model_params['batch_size'],
          callbacks = model_params['callbacks'],
          verbose = model_params['verbose'],
          validation_split = model_params['validation_split'],
          validation_data = model_params['validation_data'],
          shuffle = model_params['shuffle'],
          initial_epoch = model_params['initial_epoch'],
          steps_per_epoch = model_params['steps_per_epoch'],
          validation_steps = model_params['validation_steps'])

Epoch 1/30


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNV2' used by {{node sequential/cu_dnnlstm/CudnnRNNV2}} with these attrs: [dropout=0, seed=0, T=DT_FLOAT, input_mode="linear_input", direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0]
Registered devices: [CPU]
Registered kernels:
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_DOUBLE]

	 [[sequential/cu_dnnlstm/CudnnRNNV2]] [Op:__inference_train_function_4013]

In [ ]:
weights_file = './models/Weights-LSTM-improvement-004-2.49538-bigger.hdf5'
model.load_weights(weights_file)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [ ]:
start = np.random.randint(0, len(data_X) - 1)
pattern = data_X[start]
print('Seed:')
print("\"",''.join([int_chars[value] for value in pattern]), "\"\n")

generated_characters = 300

for i in range(generated_characters):
    x = np.reshape(pattern, (1,len(pattern), 1))
    x = x/float(n_vocab)
    prediction = model.predict(x, verbose = 0)
    index = np.argmax(prediction)
    result = int_chars[index]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print('\nDone')
